# Introduction
In this notebook we present a simple CNN architecture to classify the MAMe dataset.
With 3 Convolutional Layers and two fully connected layers we can achieve a classification accuracy of around ~~75%~~ 65%.

# Loading the dataset
The function ```load_mame``` reads the ```Mame_dataset.csv``` table and returns, for the train, validation and test subsets, a Pandas DataFrame containing the image filenames and their corresponding class (if ```dataframe=True```) or an array of filenames and a list of the corresponding classes (if ```dataframe=False```).

The DataFrame will be useful to use with Keras' ```ImageDataGenerator.flow_from_dataframe``` method.

In [ ]:
import os
import pandas as pd
import numpy as np

def load_mame(dataframe=False):
    """ Load MAMe dataset data
    Args:
      dataframe (bool): whether to return a dataframe or an array of 
                        filenames and a list of labels
      
    Returns:
      (x_train, y_train), (x_val, y_val), (x_test, y_test) if dataframe=False
      or
      df_train, df_val, df_test if dataframe=True
    """
    INPUT_PATH = '/kaggle/input/'

    # Load dataset table
    dataset = pd.read_csv(os.path.join(INPUT_PATH, 'mame-dataset', 'MAMe_dataset.csv'))
    
    # Subset divisions
    x_train_files = dataset.loc[dataset['Subset'] == 'train']['Image file'].tolist()
    y_train_class = dataset.loc[dataset['Subset'] == 'train']['Medium'].tolist()

    x_val_files = dataset.loc[dataset['Subset'] == 'val']['Image file'].tolist()
    y_val_class = dataset.loc[dataset['Subset'] == 'val']['Medium'].tolist()

    x_test_files = dataset.loc[dataset['Subset'] == 'test']['Image file'].tolist()
    y_test_class = dataset.loc[dataset['Subset'] == 'test']['Medium'].tolist()

    if dataframe:
        train = pd.DataFrame({'filename': x_train_files, 'class': y_train_class})
        val = pd.DataFrame({'filename': x_val_files, 'class': y_val_class})
        test = pd.DataFrame({'filename': x_test_files, 'class': y_test_class})
        
        # Set full path
        train['filename'] = train['filename'].transform(lambda x: INPUT_PATH + 'mame-dataset' + os.sep + 'data' + os.sep + x)
        val['filename'] = val['filename'].transform(lambda x: INPUT_PATH + 'mame-dataset' + os.sep + 'data' + os.sep + x)
        test['filename'] = test['filename'].transform(lambda x: INPUT_PATH + 'mame-dataset' + os.sep + 'data' + os.sep + x)
        
        return train, val, test
    
    else:
        # Return list of filenames
        x_train = [os.path.join(INPUT_PATH, 'mame-dataset', 'data', img_name) for img_name in x_train_files]
        x_val = [os.path.join(INPUT_PATH, 'mame-dataset', 'data', img_name) for img_name in x_val_files]
        x_test = [os.path.join(INPUT_PATH, 'mame-dataset', 'data', img_name) for img_name in x_test_files]

        return (np.array(x_train), np.array(y_train_class)), (np.array(x_val), 
              np.array(y_val_class)), (np.array(x_test), np.array(y_test_class))
    

df_train, df_val, df_test = load_mame(dataframe=True)
print(df_train.head())

## Plot examples
Let's visualize an example of each class:

In [ ]:
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

def show_images(img_df):
    """ Show images
    Plot a random sample of images for each of the class labels.
    
    Args:
      img_df (DataFrame): DataFrame with a column 'filename' with image filenames and 
                          a colum 'class' with classification labels
    """
    plt.figure(figsize=(20, 20))
    i = 1
    
    classes = img_df['class'].unique().tolist()
    for c in classes:
        # Get a random sample of an instance with class c
        filename = img_df[img_df['class']==c].sample(1)['filename'].values[0]
        
        # Plot image
        plt.subplot(6, 5, i)
        plt.imshow(load_img(filename))
        plt.title(c, fontsize=16)
        plt.axis('off')
        i += 1
                  
    plt.show()
                                                                                     
show_images(df_train)

# CNN
To classify this dataset will build a simle deep learning model based on convolutional layers. This is be done using TensorFlow and Keras.

In [ ]:
import time
import keras
import tensorflow as tf
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils


print('Using Keras version', keras.__version__)
print('Using TensorFlow version', tf.__version__)

# Initialize some variables
img_height, img_width = 256, 256
n_channels = 3
input_shape = (img_height, img_width, n_channels)
batch_size = 128

# Load dataset
df_train, df_val, df_test = load_mame(dataframe=True)
num_classes = len(df_train['class'].unique())

# Model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=input_shape, data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation=(tf.nn.softmax)))

# Print model summary
print(model.summary())

## Data augmentation and generators
We use data augmentation on the train set and create generators for the train, validation and test set.

In [ ]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        rotation_range = 30,
        zoom_range = 0.2,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        horizontal_flip = True,
        fill_mode = "nearest")

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_dataframe(
        df_train,
        target_size = (img_height, img_width),
        batch_size = batch_size, 
        class_mode = "categorical",
        validate_filenames=False)

validation_generator = test_datagen.flow_from_dataframe(
        df_val,
        target_size = (img_height, img_width),
        batch_size = batch_size,
        shuffle = False, 
        class_mode = "categorical",
        validate_filenames=False)

test_generator = test_datagen.flow_from_dataframe(
        df_test,
        target_size = (img_height, img_width),
        batch_size = 1,
        shuffle = False, 
        class_mode = "categorical",
        validate_filenames=False)



## Train the model
We compile the model using the Adam optimizer and then fit the train generator.

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001, epsilon=0.1, amsgrad=True), metrics=["accuracy"])

# Use early stopping
early = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, mode='auto', restore_best_weights=True)

# Train the model 
t0 = time.time()
epochs = 100

t0 = time.time()
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VAL=validation_generator.n//validation_generator.batch_size
    
history = model.fit(x=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VAL,
                    epochs=epochs,
                    use_multiprocessing=True,
                    workers=6,
                    callbacks = [early]
                    )
    
print('Model trained in {:.1f}min'.format((time.time()-t0)/60))

## Plot training curves
Let's plot the training and validation accurcay and loss curves.

In [ ]:
def plot_training(history):
    """ Plot training accuracy and loss curves
    
    Args:
        history (dict): history dict obtained from fit function
    """
    # Accuracy plot
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','val'], loc='upper left')
    plt.title('Training and validation accuracy')
    plt.show()
    
    # Loss plot
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','val'], loc='upper left')
    plt.title('Training and validation loss')
    plt.show()
    
plot_training(history)

## Evaluate model
Now we evaluate the model with the validation and data and output the classification report and plot a confusion matrix. After analyzing the results with the validation data, we could make decisions about our model. Only at the end we will evaluate our classifer with the test set.

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix

def evaluate_model(model, eval_gen):
    """ Evaluate given model and print results.
    Show validation loss and accuracy, classification report and 
    confusion matrix.

    Args:
        model (model): model to evaluate
        eval_gen (ImageDataGenerator): evaluation generator
    """
    # Evaluate the model
    eval_gen.reset()
    score = model.evaluate(eval_gen, verbose=0)
    print('\nLoss:', score[0])
    print('Accuracy:', score[1])
    
    # Confusion Matrix (validation subset)
    eval_gen.reset()
    pred = model.predict(eval_gen, verbose=0)

    # Assign most probable label
    predicted_class_indices = np.argmax(pred,axis=1)

    # Get class labels
    labels = (eval_gen.class_indices)
    target_names = labels.keys()

    # Plot statistics
    print(classification_report(eval_gen.classes, predicted_class_indices, target_names=target_names))

    cf_matrix = confusion_matrix(np.array(eval_gen.classes), predicted_class_indices)
    fig, ax = plt.subplots(figsize=(13, 13)) 
    sns.heatmap(cf_matrix, annot=True, cmap='PuRd', cbar=False, square=True, xticklabels=target_names, yticklabels=target_names)
    plt.show()
    
evaluate_model(model, validation_generator)

## Test the model
Finally, we evaluate the model on the test set.

In [ ]:
evaluate_model(model, test_generator)